In [43]:
from CTAFlow.data import ResultsClient, AsyncParquetWriter
from CTAFlow.strategy import HorizonMapper, ScreenerPipeline
from CTAFlow.screeners.generic import usa_quarterly_params, london_quarterly_params, us_of_all, london_of_all
from CTAFlow.screeners import HistoricalScreener, OrderflowScanner, PatternExtractor
from CTAFlow.config import INTRADAY_DATA_PATH, DLY_DATA_PATH, RESULTS_HDF_PATH
from sierrapy.parser import AsyncScidReader

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import asyncio as aio
import nest_asyncio

In [45]:
ticker_categories = {
    "grains": ["ZS", "ZC"],
    "livestock": ["HE", "LE", "GF"],
    "energy":["CL", "RB", "HO", "NG"],
    "metals": ["HG", "GC", "PA", "PL", "SI"],
}

data_mgr = AsyncParquetWriter(INTRADAY_DATA_PATH)
scid_reader = AsyncScidReader(DLY_DATA_PATH)
rc = ResultsClient(RESULTS_HDF_PATH)




In [60]:
def run_category_screen(selected_categories=None, orderflow=False,start= datetime(2020,1,1)):
    results = {}
    if selected_categories is None:
        selected_categories = ['grains', 'energy']
    nest_asyncio.apply()
    for cat in selected_categories:
        results[cat], hist, of = {}, {}, {}
        tickers = ticker_categories[cat]
        data_5m = data_mgr.batch_read_to_dict_sync(tickers, start=datetime(2020,1,1), timeframe='5min')
        hs = HistoricalScreener(data_5m, results_client=rc)
        hist['london'] = hs.run_screens(london_quarterly_params)
        hist['usa'] = hs.run_screens(usa_quarterly_params)
        hist_london = PatternExtractor(hist,hist['london'])
        hist_usa = PatternExtractor(hist,hist['usa'])
        results[cat]['london'] = hist_london
        results[cat]['usa'] = hist_usa

        if orderflow:
            screens = us_of_all
            screens.extend(london_of_all)
            of = OrderflowScanner(hdf_path=rc.results_path)
            try:
                of_res = of.run_scans(screens)
            except:pass

            else:
                results[cat]['orderflow'] = PatternExtractor(of,of_res )



    return results

In [65]:
 result = run_category_screen(['grains', 'metals', 'energy'])



2025-11-02 06:43:56,890 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO - Initialized HistoricalScreener with 2 tickers
2025-11-02 06:43:56,917 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO - Starting seasonality screen for 2 tickers
2025-11-02 06:43:56,917 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO -   Target times: ['03:00:00', '03:30:00', '02:30:00', '07:30:00', '09:30:00']
2025-11-02 06:43:56,933 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO -   Session window: 02:30:00 to 11:30:00 (America/Chicago)
2025-11-02 06:43:56,933 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO -   Filtering to months: [1, 2, 3]
Seasonality Screen: 100%|██████████| 2/2 [00:09<00:00,  4.53s/ticker]
2025-11-02 06:44:06,137 - CTAFlow.screeners.historical_screener.HistoricalScreener - INFO - Completed seasonality screen: 2/2 successful
2025-11-02 06:44:06,153 - CTAFlow.screeners.historical_screener.HistoricalScre

In [66]:
result['metals']['london'].rank_patterns()

{'HG': [('usa_q2|weekday_mean|Monday',
   PatternSummary(key='usa_q2|weekday_mean|Monday', symbol='HG', source_screen='usa_q2', screen_params=None, pattern_type='weekday_mean', description='Monday has significant mean return', strength=2.7775657050734623, payload={'type': 'weekday_mean', 'day': 'Monday', 'description': 'Monday has significant mean return', 'mean': 8.421151150194998e-05, 't_stat': 2.7775657050734623, 'p_value': 0.005492767907790297, 'strength': 2.7775657050734623}, metadata={'pattern_origin': 'strongest_patterns', 'screen_type': None}),
   0.29214315314538974),
  ('usa_q2|time_predictive_nextweek|09:00:00',
   PatternSummary(key='usa_q2|time_predictive_nextweek|09:00:00', symbol='HG', source_screen='usa_q2', screen_params=None, pattern_type='time_predictive_nextweek', description='09:00:00 predicts next week return (strongest on Thursday)', strength=0.15543529995459301, payload={'type': 'time_predictive_nextweek', 'time': '09:00:00', 'description': '09:00:00 predicts ne